In [5]:
import jax.numpy as jnp
from jax import vmap

Base Function

In [2]:
A, B, C, D = 2, 3, 4, 5
x = jnp.ones((A, B))
y = jnp.ones((B, C))
z = jnp.ones((C, D))

In [3]:
def foo(tree_arg):
  x, (y, z) = tree_arg
  return jnp.dot(x, jnp.dot(y, z))

In [32]:
tree = (x, (y, z))
print(foo(tree).shape)

(2, 5)


## (K, A, B), (B, K, C), (K, C, D) -> (A, K, D)

double `vmap`

In [28]:
K = 6
A, B, C, D = 2, 3, 4, 5
x = jnp.ones((A, B))
y = jnp.ones((B, C))
z = jnp.ones((C, D))

kx = jnp.ones((K, A, B))
ky = jnp.ones((B, K, C))
kz = jnp.ones((K, C, D))
# (K, A, B) x (B, K, C) -> (A, C, K)
vdot1 = vmap(jnp.dot, in_axes=(0, 1), out_axes=2)
# (A, C, K) x (K, C, D) -> (A, K, D)
vdot2 = vmap(jnp.dot, in_axes=(2, 0), out_axes=1)

In [31]:
kxky = vdot1(kx, ky)
print(kxky.shape)
print(vdot2(kxky, kz).shape)

(2, 4, 6)
(2, 6, 5)


`tree`

In [34]:
def foo(tree_arg):
  x, (y, z) = tree_arg
  return jnp.dot(x, jnp.dot(y, z))

In [37]:
K = 6  # batch size
x = jnp.ones((K, A, B))
y = jnp.ones((B, K, C))
z = jnp.ones((K, C, D))
tree = (x, (y, z))
vfoo = vmap(foo, in_axes=((0, (1, 0)),), out_axes=1)
print(vfoo(tree).shape)

(2, 6, 5)


결국 본질은 `pytree`